#Задание

На вебинаре мы говорили что долгое время CNN и RNN архитектуры были конурируещими выяснить какая архитектура больше подходит для задачи сантимент анализа на данных с вебинара
1. построить свёрточные архитектуры
2. построить различные архитектуры с RNN
3. построить совместные архитектуры CNN -> RNN и (RNN -> CNN)
4. сдлать выводы что получилось лучше

In [ ]:
import matplotlib.pyplot as plt

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/GB/Introduction to Natural Language Processing/lesson_8/'

In [ ]:
!pip install stop_words

  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32912 sha256=92fb6a29739f0669cbbc64ee16e3ca55a8a991e4ecc91409cf48ee930e1a416a
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [ ]:
!pip install pymorphy2
from pymorphy2 import MorphAnalyzer

     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 8.2 MB 15.7 MB/s 


In [67]:
# попробуем запрограммировать простую рекурентную сеть. Возьмем датасет с прошлого занятия

import pandas as pd
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

df_train = pd.read_csv(f"{path}/data/train.csv")
df_test = pd.read_csv(f"{path}/data/test.csv")
df_val = pd.read_csv(f"{path}/data/val.csv")

In [68]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [69]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [70]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking, RepeatVector
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from tensorflow.keras.metrics import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [71]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [72]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)

In [73]:
word_count, training_length

(258108, 27)

In [74]:
y_train = df_train['class'].values
y_val = df_val['class'].values

In [75]:
result = {'name': list(), 'score': list()}

#Общие параметры

In [76]:
batch_size = 512
epochs=10
validation_split = 0.1
verbose=1
early_stopping=EarlyStopping(monitor='val_loss')

#SimpleRNN

In [77]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [78]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=verbose,
                    validation_split=validation_split,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 28s 83ms/step - loss: 0.5629 - accuracy: 0.6959 - val_loss: 0.4976 - val_accuracy: 0.7531


In [79]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 14ms/step - loss: 0.5049 - accuracy: 0.7439


Test score: 0.5049477219581604
Test accuracy: 0.7439051270484924


In [80]:
result['name'].append('SimpleRNN')
result['score'].append(score[0])

#LSTM

Какие проблемы у рекурентных сетей?

- затухают градиенты
- медленно, нужно всегда дойти до конца

Как решить? -> LSTM


<img src="images/lstm.png">


https://colah.github.io/posts/2015-08-Understanding-LSTMs/


Давайте, кратко посмотрим как это работает:


<img src="images/LSTMMaths.png">

In [81]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(LSTM(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=verbose,
                    validation_split=validation_split,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 55s 166ms/step - loss: 0.5581 - accuracy: 0.7037 - val_loss: 0.4974 - val_accuracy: 0.7500


In [82]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 26ms/step - loss: 0.5208 - accuracy: 0.7342


Test score: 0.5208353996276855
Test accuracy: 0.7341621518135071


In [83]:
result['name'].append('LSTM')
result['score'].append(score[0])

#GRU

Какие проблемы:

- вычислительно сложно -> медленнее
- на очень длинных последовательностях все равно затухает градиент


Зачем платить больше - уберем некоторые врата (точнее совместим) -> ускоримся, уменьшим число параметров -> GRU


<img src="images/gru.png">


GRU Math


<img src="images/GRUMath.png">


In [85]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(GRU(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=verbose,
                    validation_split=validation_split,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 49s 146ms/step - loss: 0.5522 - accuracy: 0.7095 - val_loss: 0.4945 - val_accuracy: 0.7556


In [ ]:
# plt.figure(figsize=(20,12))
# plt.plot(history.history['accuracy'], '-.', label = f'accuracy', color='m')
# plt.plot(history.history['val_accuracy'], '-', label = f'val_accuracy', color='g')
        
# plt.legend()
# plt.grid('On')
# plt.title('Accuracy of epochs')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.show()

In [86]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 23ms/step - loss: 0.5059 - accuracy: 0.7475


Test score: 0.5059394240379333
Test accuracy: 0.7475201487541199


In [87]:
result['name'].append('GRU')
result['score'].append(score[0])

# CNN

In [88]:
model = Sequential()
model.add(Embedding(input_dim=word_count, 
                    input_length=training_length, 
                    output_dim=30))

model.add(Conv1D(256, 3, activation='relu'))
model.add(GlobalMaxPool1D())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='softmax'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [89]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=verbose,
                    validation_split=validation_split,
                    callbacks=[early_stopping])


Epoch 1/10
319/319 [==============================] - 33s 102ms/step - loss: 0.5444 - accuracy: 0.5076 - val_loss: 0.4871 - val_accuracy: 0.5049


In [90]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 14ms/step - loss: 0.5178 - accuracy: 0.5047


Test score: 0.5178335309028625
Test accuracy: 0.5047392249107361


In [91]:
result['name'].append('CNN')
result['score'].append(score[0])

#CNN + RNN

In [92]:
model = Sequential()
model.add(Embedding(input_dim=word_count, 
                    input_length=training_length, 
                    output_dim=30))

model.add(Conv1D(256, 3, activation='relu'))
model.add(GlobalMaxPool1D())

model.add(RepeatVector(1))

model.add(LSTM(64, recurrent_dropout=0.2))

model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='softmax'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [93]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=verbose,
                    validation_split=validation_split,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 36s 107ms/step - loss: 0.5527 - accuracy: 0.5076 - val_loss: 0.4877 - val_accuracy: 0.5049


In [94]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 15ms/step - loss: 0.5166 - accuracy: 0.5047


Test score: 0.5165602564811707
Test accuracy: 0.5047392249107361


In [95]:
result['name'].append('CNN - RNN')
result['score'].append(score[0])

#GRU + CNN

In [96]:
model = Sequential()
model.add(Embedding(input_dim=word_count, 
                    input_length=training_length, 
                    output_dim=30))

model.add(Conv1D(256, 3, activation='relu'))
model.add(Masking(mask_value=0.0))
model.add(GRU(64, recurrent_dropout=0.2))

model.add(RepeatVector(1))

model.add(GlobalMaxPool1D())

model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='softmax'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [97]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=verbose,
                    validation_split=validation_split,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 94s 287ms/step - loss: 0.5416 - accuracy: 0.5076 - val_loss: 0.4874 - val_accuracy: 0.5049


In [98]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 3s 60ms/step - loss: 0.5268 - accuracy: 0.5047


Test score: 0.5268151760101318
Test accuracy: 0.5047392249107361


In [99]:
result['name'].append('GRU + CNN')
result['score'].append(score[0])

#Result

In [100]:
df_result = pd.DataFrame(result)
df_result

,name,score
0,SimpleRNN,0.504948
1,LSTM,0.520835
2,GRU,0.505939
3,CNN,0.517834
4,CNN - RNN,0.516560
5,GRU + CNN,0.526815


#Ответ: Различные архитектуры показали близкие рез-ты.
Лучший рез-т показала архитектура - GRU+CNN - 0.526815

Остался вопрос как можно значительно улучшить рез-ты работы.

3 подхода:

<img src="images/RNNCompar.png">


Как регуляризовать?
- дропаут
- рекурентный дропаут


<img src="images/Dropouts.png">